<a href="https://colab.research.google.com/github/Nolanole/NFL-Weather-Project/blob/master/Notebook_4_QB_Data_Wrangling_%26_Merge_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports:

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

The first 3 notebooks were dealing with cleaning, merging, and wrangling w the weather data for each game. In this notebook, we'll import some football statistical data and clean/reshape and prepare it to be merged with the weather data into a master file to analyze the impacts of weather on the game.

In [0]:
#grab the url w the QB data:

url = 'https://raw.githubusercontent.com/Nolanole/NFL-Weather-Project/master/Game_Logs_Quarterback.csv'
qb = pd.read_csv(url, na_values='--')

When we previously merged weather and all_games, the all_games data included games from before 1980, so we'll need to drop those here. Also, well want to remove preseason games data as well as pro bowl games.

In [0]:
#remove any year < 1980:

qb = qb[qb['Year'] > 1979].reset_index().drop(columns=['index'])

#remove preseason and ProBowl games:

qb = qb.drop(qb[qb['Season'] == 'Preseason'].index)
qb = qb.drop(qb[qb['Season'] == 'Pro Bowl'].index).reset_index(drop=True)

#drop columns not needed: 

drop_cols = ['Position', 'Outcome','Player Id', 'Rushing Attempts', 'Rushing Yards', 'Yards Per Carry', 'Rushing TDs']
qb = qb.drop(columns=drop_cols)

#drop NaNs where no passes are attempted:
qb = qb[qb['Passes Attempted'].notna() == True]

In [0]:
#make a date column using game date + year and season type
for i in qb.index:
  if qb.loc[i]['Season'] == 'Regular Season':
    #add 1 for games of a season played in Jan/Feb of following calendar year:
    if qb.loc[i]['Game Date'] < '03/31':
      year = str(int(qb.loc[i]['Year']) + 1)
      qb.at[i, 'date'] = str(qb.loc[i]['Game Date']) + '/' + year
    else:
      qb.at[i, 'date'] = str(qb.loc[i]['Game Date']) + '/' + str(qb.loc[i]['Year'])
  else:
    year = str(int(qb.loc[i]['Year']) + 1)
    qb.at[i, 'date'] = str(qb.loc[i]['Game Date']) + '/' + year

#datetime:
qb['date'] = pd.to_datetime(qb['date'], infer_datetime_format=True)

#can safely drop Year and Game Date columns now:
qb = qb.drop(columns=['Year', 'Game Date'])

In [6]:
qb.head()

,Name,Season,Week,Home or Away,Opponent,Score,Games Played,Games Started,Passes Completed,Passes Attempted,Completion Percentage,Passing Yards,Passing Yards Per Attempt,TD Passes,Ints,Sacks,Sacked Yards Lost,Passer Rating,Fumbles,Fumbles Lost,date
13,"Savage, Tom",Regular Season,15,Home,JAX,21 to 20,1,0.0,23.0,36.0,63.9,260.0,7.2,0.0,0.0,0.0,0.0,85.4,NaN,NaN,2016-12-18
14,"Savage, Tom",Regular Season,16,Home,CIN,12 to 10,1,1.0,18.0,29.0,62.1,176.0,6.1,0.0,0.0,4.0,21.0,79.1,NaN,NaN,2016-12-24
15,"Savage, Tom",Regular Season,17,Away,TEN,17 to 24,1,1.0,5.0,8.0,62.5,25.0,3.1,0.0,0.0,1.0,15.0,67.2,1.0,1.0,2017-01-01
31,"Savage, Tom",Regular Season,15,Away,IND,10 to 17,1,0.0,10.0,19.0,52.6,127.0,6.7,0.0,1.0,1.0,5.0,51.9,2.0,1.0,2014-12-14
38,"Maddox, Tommy",Regular Season,6,Home,JAC,17 to 23,1,1.0,11.0,28.0,39.3,154.0,5.5,1.0,3.0,2.0,9.0,30.1,1.0,1.0,2005-10-16


We have enough features in common in this dataset to merge with the weather data because we can use date and opponent column. Unfortunately however, the opponent is listed by an abbreviation rather than the full name of city + mascot. In order to merge the datsets, we'll need to make a dictionary that maps the abbrev with the full team name. This was a bit of a timecrunch bc several abbreviations could apply to mutiple team names over the years, so I had to google which team played in certain cities by year. If I incorrectly renamed STL to the Rams instead of Cardinals for example, when we merge with weather df those rows will not match up

In [0]:
#get list of opponents, create dict to map with full team name so can merge with
#weather df

opponents_map = {'NE': 'New England Patriots',
                 'SF': 'San Francisco 49ers',
                 'DEN': 'Denver Broncos',
                 'PIT': 'Pittsburgh Steelers',
                 'PHI': 'Philadelphia Eagles',
                 'GB': 'Green Bay Packers',
                 'DAL': 'Dallas Cowboys',
                 'MIA': 'Miami Dolphins',
                 'CHI': 'Chicago Bears',
                 'WAS': 'Washington Redskins',
                 'NYG': 'New York Giants',
                 'SEA': 'Seattle Seahawks',
                 'MIN': 'Minnesota Vikings',
                 'CIN': 'Cincinnati Bengals',
                 'ATL': 'Atlanta Falcons',
                 'NO': 'New Orleans Saints',
                 'KC': 'Kansas City Chiefs',
                 'BUF': 'Buffalo Bills',
                 'TB': 'Tampa Bay Buccaneers',
                 'NYJ': 'New York Jets',
                 'DET': 'Detroit Lions',
                 'IND': 'Indianapolis Colts',
                 'CLE': 'Cleveland Browns',
                 'ARI': 'Arizona Cardinals',
                 'CAR':'Carolina Panthers',
                 'JAC': 'Jacksonville Jaguars',
                 'JAX': 'Jacksonville Jaguars',
                 'SD': 'San Diego Chargers',
                 'PHO': 'Phoenix Cardinals',
                 'LA': 'Los Angeles Rams',
                 'RAM': 'Los Angeles Rams',
                 'RAI': 'Los Angeles Raiders', #1982-1994
                 'OAK': 'Oakland Raiders'} 

qb['Opponent'] = qb['Opponent'].replace(opponents_map)


#Replace TEN with tennessee oilers or titans, depending on year:
tenn_i = qb[(qb['Opponent']=='TEN')].index
for i in tenn_i:
  if qb.loc[i].date.year < 1999:
    qb.at[i, 'Opponent'] = 'Tennessee Oilers'
  else:
    qb.at[i, 'Opponent'] = 'Tennessee Titans'

#Same for BAL: Colts until 1984, Ravens after 1996
balt_i = qb[(qb['Opponent']=='BAL')].index
for i in balt_i:
  if qb.loc[i].date.year < 1986:
    qb.at[i, 'Opponent'] = 'Baltimore Colts'
  else:
    qb.at[i, 'Opponent'] = 'Baltimore Ravens'

#Same for HOU: Oilers until 1996, Texans after 2002
houston_i = qb[(qb['Opponent']=='HOU')].index
for i in houston_i:
  if qb.loc[i].date.year < 1998:
    qb.at[i, 'Opponent'] = 'Houston Oilers'
  else:
    qb.at[i, 'Opponent'] = 'Houston Texans'

#Same for STL: Cardinals until 1986, Rams 1995-2016
stl_i = qb[(qb['Opponent']=='STL')].index
for i in stl_i:
  if qb.loc[i].date.year < 1988:
    qb.at[i, 'Opponent'] = 'St. Louis Cardinals'
  else:
    qb.at[i, 'Opponent'] = 'St. Louis Rams'

Now that we have our opponent column in the correct format to match the weather df, we have one more step before we can merge:
The weather dataframe has the teams listed under 'home' and 'away' columns. But the QB data is a mix of data from home and away, but only includes one team, the opponent- for some reason the team the player plays for is not included. Oh well, we can work around this bc QB data has a column 'home or away' which indicates if the QB is on the home or away team. Since we have that info, we can extrapolate the reciprocal and assign opponent column as either home or away.
We can split the QB df into 2 dataframes, one for home QBs and one for away QBs, and then merge each with the weather data, and then later combine them back into a final master dataframe with all QB data and weather data.

In [0]:
'''Split into 2 DFs, one for home teams and one for away: then make opponent column 
correspond to home or away so we can merge with weather data using opponent and date'''

home_qb = qb[qb['Home or Away'] == 'Home'].reset_index(drop=True)
away_qb = qb[qb['Home or Away'] == 'Away'].reset_index(drop=True)

#change the opponent column to home or away:

for i in home_qb.index:
  home_qb.at[i, 'away'] = home_qb.at[i, 'Opponent']

for i in away_qb.index:
  away_qb.at[i, 'home'] = away_qb.at[i, 'Opponent']

In [0]:
#get the weather games data into df:
weather_data_url = 'https://raw.githubusercontent.com/Nolanole/NFL-Weather-Project/master/all_games_weather.csv'
weather_df = pd.read_csv(weather_data_url)

In [0]:
#convert date to datetime:
weather_df['date'] = pd.to_datetime(weather_df['date'], infer_datetime_format=True)

In [0]:
#merge with weather data:
home_weather = pd.merge(home_qb, weather_df, how='outer', on=['date', 'away'])
away_weather = pd.merge(away_qb, weather_df, how='outer', on=['date', 'home'])

In [0]:
#Drop rows where game_id is NaN

home_weather = home_weather[home_weather.game_id.isna()==False].reset_index(drop=True)
away_weather = away_weather[away_weather.game_id.isna()==False].reset_index(drop=True)

#Drop rows where qb stats are NaN:

home_weather = home_weather[home_weather.Name.isna()==False].reset_index(drop=True)
away_weather = away_weather[away_weather.Name.isna()==False].reset_index(drop=True)

In [0]:
#rename columns
drop_cols= ['Unnamed: 0','Games Played', 'Games Started'] 
col_mapper = {'Name':'name', 'Season':'season', 'Week':'week', 'Home or Away':'home_or_away',
              'Score':'score','Passes Completed':'pass_completions', 'Passes Attempted':'pass_attempts', 
              'Completion Percentage':'comp_percent', 'Passing Yards':'yards', 
              'Passing Yards Per Attempt':'ypa', 'TD Passes':'tds', 'Ints':'ints', 
              'Sacks':'sacks', 'Sacked Yards Lost':'sack_yards', 'Passer Rating':'rating', 
              'Fumbles':'fumbles', 'Fumbles Lost':'fumbles_lost', 'Opponent':'opponent'}

home_weather.drop(columns=drop_cols, inplace=True)
away_weather.drop(columns=drop_cols, inplace=True)

home_weather.rename(columns=col_mapper, inplace=True)
away_weather.rename(columns=col_mapper, inplace=True)

In [0]:
#recast game_id as int:
home_weather['game_id'] = home_weather['game_id'].astype('int64')
away_weather['game_id'] = away_weather['game_id'].astype('int64')

Because our end goal is a dataframe where each row contains a one full game game worth of data for a teams QB, the data will be innaccurate for all games where the starting QB got injured or benched and did not play the full game. If he is hurt in the first quarter and only attempted 3 passes for 30 yards, that will show up as an extreme outlier and skew the overall data. Since we have a game_id identifier for each observation, we can aggregate by that feature and, similarly to how we combined rows for the hourly weather data into a single row for each game, can aggregate total yards and other QB statistics for that game. It wont be 100% accurate for each QB, but the overall summary statistics on a per game basis should b accurate:

In [0]:
#combine stats where multiple qbs played in same game:
home_vc = home_weather.game_id.value_counts()
away_vc = away_weather.game_id.value_counts()

#3_qbs in same game:
home_ids_3 = list(home_vc[home_vc == 3].index)
away_ids_3 = list(away_vc[away_vc == 3].index)

#2 Qbs same game:
home_ids_2 = list(home_vc[home_vc == 2].index)
away_ids_2 = list(away_vc[away_vc == 2].index)

#combine to one list:
home_ids = home_ids_3 + home_ids_2
away_ids = away_ids_3 + away_ids_2

In [0]:
cols_to_keep = ['season', 'week', 'home_or_away', 'opponent', 'score', 
                'date', 'away', 'home', 'score_home', 'score_away', 'game_id', 
                'stadium', 'fog_or_haze', 'avg_temp', 'avg_dewpoint', 'avg_humidity', 
                'avg_wind', 'max_windgust', 'windchill', 'windchill_gust', 'sky', 
                'precipitation']

home_new_rows = []

for id in home_ids:
  game = home_weather[home_weather.game_id == id]
  
  name = game.name.max()
  pass_completions = game.pass_completions.sum()
  pass_attempts = game.pass_attempts.sum()
  yards = game.yards.sum()
  tds = game.tds.sum()
  ints = game.ints.sum()
  sacks = game.sacks.sum()
  sack_yards = game.sack_yards.sum()
  fumbles = game.fumbles.sum()
  fumbles_lost = game.fumbles_lost.sum()
  ypa = yards/pass_attempts
  comp_percent = (pass_completions / pass_attempts)*100
  rating = np.nan
  
  game_row = game[cols_to_keep]
  game_row['name'] = name
  game_row['pass_completions'] = pass_completions
  game_row['pass_attempts'] = pass_attempts
  game_row['yards'] = yards
  game_row['tds'] = tds
  game_row['ints'] = ints
  game_row['sacks'] = sacks
  game_row['sack_yards'] = sack_yards
  game_row['fumbles'] = fumbles
  game_row['fumbles_lost'] = fumbles_lost
  game_row['ypa'] = ypa
  game_row['comp_percent'] = comp_percent
  game_row['rating'] = rating
  home_new_rows.append(game_row.iloc[0,:])



In [0]:
#repeat for away df:  

away_new_rows = []

for id in away_ids:
  game = away_weather[away_weather.game_id == id]
  
  name = game.name.max()
  pass_completions = game.pass_completions.sum()
  pass_attempts = game.pass_attempts.sum()
  yards = game.yards.sum()
  tds = game.tds.sum()
  ints = game.ints.sum()
  sacks = game.sacks.sum()
  sack_yards = game.sack_yards.sum()
  fumbles = game.fumbles.sum()
  fumbles_lost = game.fumbles_lost.sum()
  ypa = yards/pass_attempts
  comp_percent = (pass_completions / pass_attempts)*100
  rating = np.nan
  
  game_row = game[cols_to_keep]
  game_row['name'] = name
  game_row['pass_completions'] = pass_completions
  game_row['pass_attempts'] = pass_attempts
  game_row['yards'] = yards
  game_row['tds'] = tds
  game_row['ints'] = ints
  game_row['sacks'] = sacks
  game_row['sack_yards'] = sack_yards
  game_row['fumbles'] = fumbles
  game_row['fumbles_lost'] = fumbles_lost
  game_row['ypa'] = ypa
  game_row['comp_percent'] = comp_percent
  game_row['rating'] = rating
  away_new_rows.append(game_row.iloc[0,:])

In [0]:
#concat the new rows of series objects (rows, transposed):
new_home_rows = pd.concat(home_new_rows, axis=1).T.reset_index().drop(columns=['index'])
new_away_rows = pd.concat(away_new_rows, axis=1).T.reset_index().drop(columns=['index'])

Get the indices of the rows we just aggregated into new rows, so we can drop them from the main dataframes by index, and then we'll add the new rows of aggregated games data back in and reset the index



In [0]:
#indices of game_ids where 3 QB played:
i_home_3 = list(home_weather[home_weather.game_id.isin(home_ids_3)].index)
i_away_3 = list(away_weather[away_weather.game_id.isin(away_ids_3)].index)

#indices of game_ids where 2 QB played:
i_home_2 = list(home_weather[home_weather.game_id.isin(home_ids_2)].index)
i_away_2 = list(away_weather[away_weather.game_id.isin(away_ids_2)].index)

#combined
home_indices = i_home_3 + i_home_2
away_indices = i_away_3 + i_away_2

In [0]:
#drop these rows from home and away weather dfs:

home_weather.drop(home_indices, inplace=True)
away_weather.drop(away_indices, inplace=True)

In [0]:
#concat new rows with home and away_weather dfs:
home_plus = pd.concat([home_weather, new_home_rows])
away_plus = pd.concat([away_weather, new_away_rows])

In [0]:
#concat home and away
combined = pd.concat([home_plus, away_plus])

In [0]:
#drop rows where pass attempts == 0
combined = combined[combined.pass_attempts > 0]

In [0]:
#sort by date/ stadium
combined = combined.sort_values(['date', 'stadium']).reset_index(drop=True)

In [0]:
#add categorical column for dome, and fill some nan for max_windgust

for i in combined.index:
  if combined.at[i, 'stadium'] == 'dome':
    combined.at[i, 'dome'] = 1
  else:
    combined.at[i, 'dome'] = 0
  if np.isnan(combined.at[i, 'max_windgust'])==True:
    if np.isnan(combined.at[i, 'avg_wind']) == False:
      combined.at[i, 'max_windgust'] = combined.at[i, 'avg_wind']

In [0]:
#export to csv and start new notebook for visualizations
#save and export:
combined.to_csv('QB_weather_cleaned.csv')

#download the csv:
from google.colab import files
files.download('QB_weather_cleaned.csv')